In [ ]:
import pandas as pd
import numpy as np

# scrape all the tables from the web
all_df_lists = []
for year in range(2014, 2020):
    url='http://www.svpl.org.uk/index.php/{}-league-tables?year={}-league-tables'.format(year, year)
    all_df_lists.append(pd.read_html(url))

In [ ]:
all_df_lists

In [ ]:
## not all tables have all desired columns, so fill with NaN
# also add columns for Year and table number within year

desired_columns = ['Year', 'TblNbr', 'Rank', 'Teams', 'Played', 'Points', 'GD', 'PD']
for year, df_list in zip(range(2014, 2020), all_df_lists):
    for i, df in enumerate(df_list):
        df['Year'] = year
        df['TblNbr'] = i
        for col in desired_columns:
            if not col in df.columns:
                df[col] = np.nan
                print("added column {} to table {} of year {}".format(col, i, year))
desired_columns

In [ ]:
df_list

In [ ]:
# combine the tables, keep only the desired columns
df_combined = pd.concat([df[desired_columns] for df_list in all_df_lists for df in df_list], ignore_index=True)
df_combined

In [ ]:
#teams=['Kinver Con Club','Nomads','Top Pub','Plough','Alveley Royals','Footloose',"Gaters 'A'",'Alveley Oaks']
final=df_combined[(df_combined.values == 'Alveley Oaks')|(df_combined.values == 'Kinver Con Club')|(df_combined.values == 'Nomads')|(df_combined.values == 'Kinver Con Club')|(df_combined.values == 'Top Pub')|(df_combined.values == 'Plough')|(df_combined.values == 'Alveley Royals')|(df_combined.values == 'Footloose')|(df_combined.values == "Gaters 'A'")|(df_combined.values == 'Alveley Oaks')]
#df[(df.values  == "banana")|(df.values  == "apple" ) ]
final=final[['Year','Teams','Points']]
merged=final.pivot_table(index='Year', columns='Teams',values='Points').reset_index().rename_axis(None, axis=1)
merged.Year = pd.to_datetime(merged['Year'], format='%Y')
merged.set_index(['Year'],inplace=True)
merged

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.transforms as mtrans
from matplotlib.ticker import FixedLocator, FormatStrFormatter
import matplotlib.dates as mdates
%matplotlib notebook
plt.style.use('seaborn-poster')

fig,ax = plt.subplots(figsize=(9,6), facecolor='w', edgecolor='blue')

my_colors=['orangered','seagreen', 'goldenrod', 'springgreen', 'sienna']

ax.set_ylim(-0.7, 31)  

# set every year
years = mdates.YearLocator()   
years_fmt = mdates.DateFormatter('%Y')

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)


ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)


#title

fig.text(0.24, 0.95, "S", ha="center", va="bottom", size=21,color="blue")
fig.text(0.27, 0.95, "V", ha="center", va="bottom", size=21,color="red")
fig.text(0.30, 0.95, "P", ha="center", va="bottom", size=21,color="gold")
fig.text(0.33, 0.95, "L", ha="center", va="bottom", size=21,color="darkgreen")
fig.text(0.50, 0.95, "Clubs  performance", ha="center", va="bottom", size=17,color="black")
fig.text(0.73, 0.95," 2014-2019", ha="center", va="bottom", size=17,color="black")

x_axis = ax.axes.get_xaxis()
x_label = x_axis.get_label()
x_label.set_visible(False)
ax.set_ylabel('Points won', color='darkblue',  fontdict={'fontsize': 14, 'fontweight': 'medium'})



ax.tick_params(axis='y', which='major', labelsize=12, bottom=False, top=False, labelbottom=False,
               left=False, right=False, labelright=False)
ax.tick_params(axis='x', which='major', labelcolor='black', width=3,color='black', labelsize=14, bottom=True, top=False, labelbottom=True, labelrotation=90,
               left=False, right=False, labelright=False)

ax=merged.plot(y=['Kinver Con Club', 'Nomads', 'Plough', "Gaters 'A'", 'Alveley Oaks'],marker='o',figsize=(9,6),legend=None, color=my_colors, ax=ax)

selectcols=merged[['Kinver Con Club', 'Nomads', 'Plough', "Gaters 'A'", 'Alveley Oaks']]

for line, name in zip(ax.lines, selectcols.columns):

    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(0.998,y), xytext=(6,0), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=16, va="center")

merged.plot(y='Footloose', marker='o', figsize=(9,6), legend=None, color= 'steelblue', ax=ax, alpha=1,
                    transform=mtrans.offset_copy(ax.transData, fig=fig, x=0.0, y=4, units='points'))   

y = 12
ax.annotate('Footloose', xy=(0.998,y), xytext=(6,0), color='steelblue', 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=16, va="center")

merged.plot(y=['Top Pub'], marker='o', figsize=(9,6), legend=None, color= 'darkred', ax=ax, alpha=1,
                    transform=mtrans.offset_copy(ax.transData, fig=fig, x=0.0, y=3*(1), units='points'))      
y = 22
ax.annotate('Top Pub', xy=(0.998,y), xytext=(6,0), color='darkred', 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=16, va="center")

y = 16
merged.plot(y='Alveley Royals', marker='o', figsize=(9,6), legend=None, color= 'darkviolet', ax=ax, alpha=1,
                    transform=mtrans.offset_copy(ax.transData, fig=fig, x=0.0, y=-1.8, units='points'))  
ax.annotate('Alveley Royals', xy=(0.998,y), xytext=(6,0), color='darkviolet', 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=16, va="center")


# Adjusting the size of the plot to fit the team labels

fig.subplots_adjust(left=.075, right=.72, bottom=0.15, top=.90)

# Horizontal grid lines
ax.yaxis.set_major_formatter(plt.FuncFormatter('{:.0f}'.format))
ax.grid(True, 'major', 'y', ls='--', lw=.6, c='darkgray', alpha=.5)

plt.savefig('SVPLclubsperformancebest.pdf')
plt.show()
